# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
#import
# import pandas as pdnp
import numpy as np


# Challenge 1 - Loading and Evaluating The Data

In this lab, we will look at a dataset of sensor data from a cellular phone. The phone was carried in the subject's pocket for a few minutes while they walked around.

To load the data, run the code below.

In [14]:

# Run this code:

sensor = pd.read_csv('../sub_1.csv')
sensor.drop(columns=['Unnamed: 0'], inplace=True)
sensor

Examine the data using the `head` function.

Check whether there is any missing data. If there is any missing data, remove the rows containing missing data.

In [23]:
sensor.values

array([[ 1.528132, -0.733896,  0.696372, ...,  0.294894, -0.184493,
         0.377542],
       [ 1.527992, -0.716987,  0.677762, ...,  0.219405,  0.035846,
         0.114866],
       [ 1.527765, -0.706999,  0.670951, ...,  0.010714,  0.134701,
        -0.167808],
       ...,
       [ 1.830821, -0.578367,  2.447967, ..., -0.08559 , -0.030209,
        -0.08774 ],
       [ 1.849557, -0.586962,  2.439458, ..., -0.048105,  0.029555,
         0.060441],
       [ 1.869375, -0.596783,  2.433775, ..., -0.065011, -0.042575,
         0.046052]])

In [22]:
if not sensor.isnull().values.any():
    print("No hay valores null")

No hay valores null


How many rows and columns are in our data?

In [27]:
(fil,col)=sensor.shape

print(f"Tiene {fil} filas y {col} columnas")

Tiene 1751 filas y 12 columnas


To perform time series analysis on the data, we must change the index from a range index to a time series index. In the cell below, create a time series index using the `pd.date_range` function. Create a time series index starting at 1/1/2018 00:00:00 and ending at 1/1/2018 00:29:10. The number of periods is equal to the number of rows in `sensor`. The frequency should be set to `infer`.

In [5]:
pd.date_range()


Assign the time series index to the dataframe's index.

In [6]:
# Your code here:



Our next step is to decompose the time series and evaluate the patterns in the data. Load the `statsmodels.api` submodule and plot the decomposed plot of `userAcceleration.x`. Set `freq=60` in the `seasonal_decompose` function. Your graph should look like the one below.

![time series decomposition](../images/tsa_decompose.png)

In [7]:
# Your code here:



Plot the decomposed time series of `rotationRate.x` also with a frequency of 60.

In [7]:
# Your code here:



Let's plot the `lag_plot` to detect any autocorrelation. Do this for `userAcceleration.x`

In [8]:
# Your code here:



Create a lag plot for `rotationRate.x`

In [10]:
# Your code here:



What are your conclusions from both visualizations?

In [11]:
# Your conclusions here:



The next step will be to test both variables for stationarity. Perform the Augmented Dickey Fuller test on both variables below.

In [12]:
# Your code here:



What are your conclusions from this test?

In [13]:
# Your conclusions here:

